# Models, clean code

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPooling3D
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping

from keras.applications import MobileNet

import numpy as np
import os
import cv2
import glob

In [ ]:
train_dir = 'cnn_train'
train_path = os.path.join(train_dir, '*g')

train_imgsNom = glob.glob(train_path)
train_labels = [int(name[-5]) for name in train_imgsNom]
train_imgs = np.array([np.array(cv2.imread(img)) for img in train_imgsNom])
# train_imgs = train_imgs.reshape(tuple(np.append(train_imgs.shape, 1)))

train_imgs.shape

In [ ]:
test_dir = 'cnn_test'
test_path = os.path.join(test_dir, '*g')

test_imgsNom = glob.glob(test_path)
test_labels = np.array([int(name[-5]) for name in test_imgsNom])
test_imgs = np.array([np.array(cv2.imread(img)) for img in test_imgsNom])
# test_imgs = test_imgs.reshape(tuple(np.append(test_imgs.shape, 1)))

test_imgs.shape

For image loading and preprocessing see:
https://keras.io/preprocessing/image/

In [ ]:
num_classes = 2

# input image dimensions
img_rows, img_cols = 60, 60

print('train_imgs shape:', train_imgs.shape)
print(train_imgs.shape[0], 'train samples')
print(test_imgs.shape[0], 'test samples')

# convert class vectors to binary class matrices
train_label = keras.utils.to_categorical(train_labels, num_classes)
# val_label = keras.utils.to_categorical(val_labels, num_classes)
test_label = keras.utils.to_categorical(test_labels, num_classes)


In [ ]:
# mobile = 

In [ ]:
### Basic CNN ###

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', 
                 input_shape=(img_rows,img_cols,3)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer = 'random_normal'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Total params: 6,441,730
# Na 30 epochs: 0.69158878727494

In [ ]:
### The model used by DNT ###

model2 = Sequential()
model2.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model2.add(Conv2D(5, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model2.add(Dropout(0.3))
model2.add(Flatten())
model2.add(Dense(24*24*5, activation='relu', kernel_initializer = 'random_normal'))
model2.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Total params: 41,999,531
# Na 30 epochs 0.69158878727494

# Profile:
# node name | # float_ops
# Mul                      339.10m float_ops (100.00%, 35.00%)
# Add                      242.22m float_ops (65.00%, 25.00%)
# Maximum                  96.88m float_ops (40.00%, 10.00%)
# Minimum                  96.88m float_ops (30.00%, 10.00%)
# Square                   96.88m float_ops (20.00%, 10.00%)
# Sub                      48.44m float_ops (10.00%, 5.00%)
# RealDiv                  48.44m float_ops (5.00%, 5.00%)
# AssignAdd                    2 float_ops (0.00%, 0.00%)

In [ ]:
### Our improvement of DNT ###

model3 = Sequential()
model3.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model3.add(Conv2D(5, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model3.add(MaxPooling2D(pool_size=(4,4), padding='valid'))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(16, activation='relu', kernel_initializer = 'random_normal'))
model3.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Total params: 14,059
# Na 30 epochs: 0.7149532687998263

# Profile:
# node name | # float_ops
# Mul                      339.22m float_ops (100.00%, 35.00%)
# Add                      242.30m float_ops (65.00%, 25.00%)
# Maximum                  96.91m float_ops (40.00%, 10.00%)
# Minimum                  96.91m float_ops (30.00%, 10.00%)
# Square                   96.91m float_ops (20.00%, 10.00%)
# Sub                      48.46m float_ops (10.00%, 5.00%)
# RealDiv                  48.46m float_ops (5.00%, 5.00%)
# AssignAdd                    3 float_ops (0.00%, 0.00%)

# 969167819 flops?


In [ ]:
### B-Human ball-detector model ###

model4 = Sequential()
model4.add(BatchNormalization())
model4.add(Conv2D(4, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model4.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model4.add(BatchNormalization())
model4.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
#                  input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model4.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model4.add(BatchNormalization())
model4.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
#                  input_shape=(img_rows,img_cols,3),
                 strides=(2, 2),
                 kernel_initializer = 'random_normal'))
model4.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model4.add(Flatten())
model4.add(BatchNormalization())
model4.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Total params: 1,486
# Na 30 epochs 0.9112149515998698

# Profile:
# node name | # float_ops
# Mul                      339.23m float_ops (100.00%, 35.00%)
# Add                      242.31m float_ops (65.00%, 25.00%)
# Maximum                  96.91m float_ops (40.00%, 10.00%)
# Square                   96.91m float_ops (30.00%, 10.00%)
# Minimum                  96.91m float_ops (20.00%, 10.00%)
# Sub                      48.46m float_ops (10.00%, 5.00%)
# RealDiv                  48.46m float_ops (5.00%, 5.00%)
# AssignSub                  348 float_ops (0.00%, 0.00%)
# AddN                       318 float_ops (0.00%, 0.00%)
# Neg                        144 float_ops (0.00%, 0.00%)
# Rsqrt                      144 float_ops (0.00%, 0.00%)
# AssignAdd                   12 float_ops (0.00%, 0.00%)
# Pow                          8 float_ops (0.00%, 0.00%)

# 969196474

In [ ]:
### B-Human ball-detector model with max pooling first###

model5 = Sequential()
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(4, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
#                  input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
#                  input_shape=(img_rows,img_cols,3),
                 strides=(2, 2),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(Flatten())
model5.add(BatchNormalization())
model5.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Total params: 1,102
# Na 30 epochs 0.8224299093273199

# Profile:
# node name | # float_ops
# Mul                      339.23m float_ops (100.00%, 35.00%)
# Add                      242.32m float_ops (65.00%, 25.00%)
# Maximum                  96.92m float_ops (40.00%, 10.00%)
# Square                   96.92m float_ops (30.00%, 10.00%)
# Minimum                  96.92m float_ops (20.00%, 10.00%)
# Sub                      48.46m float_ops (10.00%, 5.00%)
# RealDiv                  48.46m float_ops (5.00%, 5.00%)
# AssignSub                  440 float_ops (0.00%, 0.00%)
# AddN                       380 float_ops (0.00%, 0.00%)
# Neg                        160 float_ops (0.00%, 0.00%)
# Rsqrt                      160 float_ops (0.00%, 0.00%)
# AssignAdd                   21 float_ops (0.00%, 0.00%)
# Pow                         16 float_ops (0.00%, 0.00%)


In [ ]:
### B-Human 3###

model5 = Sequential()
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
#                  input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
#                  input_shape=(img_rows,img_cols,3),
                 strides=(2, 2),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(Flatten())
model5.add(BatchNormalization())
model5.add(Dense(128, activation='relu', kernel_initializer = 'random_normal'))
model5.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Total params: 2,910
# Test accuracy: 0.9065420588600301

In [ ]:
### B-Human 4###

model5 = Sequential()
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(BatchNormalization())
model5.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 strides=(2, 2),
                 kernel_initializer = 'random_normal'))
model5.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model5.add(Flatten())
model5.add(BatchNormalization())
model5.add(Dense(128, activation='relu', kernel_initializer = 'random_normal'))
model5.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

In [ ]:
batch_size = 128
epochs = 40
# early_stopping = EarlyStopping(monitor='val_loss', patience=2)

current_model = model5

current_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# current_model.fit(train_imgs, train_label,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_split=.2,
#           callbacks=[early_stopping])

current_model.fit(train_imgs, train_label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=.2)

score = current_model.evaluate(test_imgs, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

current_model.summary()

In [ ]:
# Save current model 1705850201, 1706099677
# current_model.save('current_cnn.h5')

# Speed measures, not clean code

In [ ]:
# run_meta = tf.RunMetadata()
# with tf.Session(graph=tf.Graph()) as sess:
#     keras.backend.set_session(sess)
#     net = model5

#     opts = tf.profiler.ProfileOptionBuilder.float_operation()    
#     flops = tf.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)
# # 
#     opts = tf.profiler.ProfileOptionBuilder.trainable_variables_parameter()    
#     params = tf.profiler.profile(sess.graph, run_meta=run_meta, cmd='op', options=opts)

#     print("{:,} --- {:,}".format(flops.total_float_ops, params.total_parameters))

In [ ]:
import time

In [ ]:
start = time.time()
current_model.predict(test_imgs)
end = time.time()
print(end - start)

In [ ]:
start = time.perf_counter()
current_model.predict(test_imgs)
end = time.perf_counter()
execution_time = (end - start)
execution_time

In [ ]:
import timeit

In [ ]:
start = timeit.default_timer()
current_model.predict(test_imgs)
end = timeit.default_timer()
print(end - start)

In [ ]:
import tensorflow as tf
import keras.backend as K

In [ ]:
def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=K.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops  # Prints the "flops" of the model.

In [ ]:
print(get_flops(current_model))

# Leons code

In [ ]:
from texttable import Texttable

pred_test =  current_model.predict(test_imgs)
predT = np.array(pred_test[:,0]) < 0.5
realT = test_labels == 1
print(sum(predT == realT))
print(len(predT))
print("accuracy is:",accuracy(predT,realT))
# print(predT)
# print(realT)
def accuracy(predT, realT):
    return sum(predT == realT)/ len(realT)    

def precision(predT,realT):
    # if it says robot is it robot?
    return sum(predT and realT)/predT

# def recall()

def evalMatrix(predT,realT):
    t = Texttable()
    t.add_rows([['Pred\Real', 'True', 'False '], ['True', sum(predT & realT), sum(predT & (~realT)) ], ['False',  sum(~predT & realT), sum(~ predT & ~ realT) ]])
    print(t.draw())
evalMatrix(np.array(predT),np.array(realT))
# print(~realT)

In [ ]:
leon = np.array([True,True,False,False])
Hm = np.array([True,False,False,True])
print(leon & Hm)
print(165/216)

In [ ]:
# model2.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model2.fit(train_imgs, train_label,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_split=.2)
# score = model2.evaluate(test_imgs, test_label, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.array(test_imgsNom)[predT & (~realT)]
# print(test_imgsNom)

In [ ]:
print(test_labels)
plt.imshow(test_imgs[0])